In [1]:
from data_helper import *
from obj_helper import *
from algo import *
from approx import *
import pickle

Using Python-MIP package version 1.5.1


### Large instance: Influence Maximization on Youtube

In [2]:
### Example of influence maximization on Youtube data

# params
k = 10
fname = 'youtube'

# open weights and graph
G = open_data("youtube_graph.p")
ground_set = set(range(len(G)))

# get objective
obj = get_obj('influence_maximization', G)

# query objective
print(obj([225]))
print(obj([225, 428]))

76
132


In [3]:
### run submodular maximization algorithms
# each function outputs the set chosen and the solution value for each k

# run greedy
res_gr = run_greedy(k, ground_set, obj, fname=fname, print_log=False)
print('Greedy')
print(res_gr, '\n')

# run local search
res_ls = run_local_search(k, ground_set, obj, fname=fname, every_iter=1)
print('Local Search')
print(res_ls, '\n')

# run random greedy
res_gr_random = run_random_greedy(k, ground_set, obj, fname=fname, every_iter=1)
print('Random Greedy')
print(res_gr_random, '\n')

# run lazier than lazy greedy
res_gr_lazier = run_lazier_greedy(k, ground_set, obj, eps=0.01, fname=fname, every_iter=1)
print('Lazier Greedy')
print(res_gr_lazier, '\n')

Greedy
[([], 0), ([225], 76), ([428], 132), ([134], 178), ([614], 216), ([45], 251), ([598], 283), ([554], 310), ([496], 334), ([986], 358), ([246], 377)] 

Local Search
[([], 0), ([225], 76), ([225, 428], 132), ([225, 428, 134], 178), ([225, 428, 134, 614], 216), ([225, 428, 45, 614, 134], 251), ([225, 428, 554, 598, 614, 45], 284), ([225, 428, 134, 554, 598, 614, 45], 310), ([225, 428, 134, 554, 614, 45, 496, 598], 334), ([225, 428, 554, 614, 45, 496, 598, 986, 134], 358), ([225, 428, 554, 614, 45, 496, 598, 986, 134, 246], 377)] 

Random Greedy
[([], 0), ([225], 76), ([225, 134], 122), ([225, 134, 428], 178), ([134, 614, 225, 554], 190), ([428, 134, 269, 45, 598], 202), ([598, 428, 134, 614, 45, 269], 239), ([225, 609, 45, 135, 614, 134, 554], 268), ([614, 609, 45, 428, 554, 201, 225, 598], 307), ([428, 17, 134, 269, 614, 45, 286, 554, 236], 287), ([17, 236, 293, 225, 134, 135, 986, 609, 496, 45], 291)] 

Lazier Greedy
[([], 0), ([225], 76), ([225, 428], 132), ([225, 428, 134], 178)

In [4]:
### run methods proposed in paper

# run method 1
ub, approx = compute_method1(k, obj, ground_set, res_gr)
print('Method 1')
print('OPT upper bound: ', ub)
print('Greedy approximation: ', approx, '\n')

# run method 3
ub, approx = compute_method3(k, obj, ground_set, res_gr)
print('Method 3')
print('OPT upper bound: ', ub)
print('Greedy approximation: ', approx, '\n')

# run dual
# S_idx specifies the greedy solution sets to comprise mathcal S
ub, approx = compute_dual(k, obj, ground_set, res_gr, S_idx=[0,1,2])
print('Dual')
print('OPT upper bound: ', ub)
print('Greedy approximation: ', approx, '\n')

Method 1
OPT upper bound:  [76, 132, 178, 222, 263, 300, 335, 366, 395, 423]
Greedy approximation:  [1.0, 1.0, 1.0, 0.972972972972973, 0.9543726235741445, 0.9433333333333334, 0.9253731343283582, 0.912568306010929, 0.9063291139240506, 0.8912529550827423] 

Method 3
OPT upper bound:  [76, 132, 178, 220, 260, 296, 328, 359, 388, 415]
Greedy approximation:  [1.0, 1.0, 1.0, 0.9818181818181818, 0.9653846153846154, 0.956081081081081, 0.9451219512195121, 0.9303621169916435, 0.9226804123711341, 0.908433734939759] 

Dual
OPT upper bound:  [76.0, 132.0, 178.0, 220.0, 260.0, 296.0, 328.0, 359.0, 388.0, 415.0]
Greedy approximation:  [1.0, 1.0, 1.0, 0.9818181818181818, 0.9653846153846154, 0.956081081081081, 0.9451219512195121, 0.9303621169916435, 0.9226804123711341, 0.908433734939759] 



In [5]:
### run benchmarks
# the following functions return the approximation to Greedy

# run top-k and output OPT upper bound and approximation
ub, approx = compute_topk(k, obj, ground_set, res_gr)
print('Top-k')
print('OPT upper bound: ', ub)
print('Greedy approximation: ', approx, '\n')

# run marginal and output OPT upper bound and approximation
# note that approximation improves when res_gr contains 
# the solutions for larger k
ub, approx = compute_marginal(k, res_gr)
print('Marginal')
print('OPT upper bound: ', ub)
print('Greedy approximation: ', approx, '\n')

# run curvature and output approximation
res = compute_curvature_ub(k, ground_set, obj)
print('Curvature Upper Bound')
print(res, '\n')

# run integer program to compute OPT exactly and output the optimal set and value
# the following can be slow
res = run_opt_ip(k, G, np.ones(len(G)))
print('IP')
print(res)

Top-k
OPT upper bound:  [76, 132, 178, 222, 264, 304, 340, 375, 407, 439]
Greedy approximation:  [1.0, 1.0, 1.0, 0.972972972972973, 0.9507575757575758, 0.930921052631579, 0.9117647058823529, 0.8906666666666667, 0.8796068796068796, 0.8587699316628702] 

Marginal
OPT upper bound:  [76.0, 152.0, 228.00000000000006, 300.0, 356.0, 406.00000000000006, 444.0000000000001, 482.0, 520.0, 548.0000000000002]
Greedy approximation:  [1.0, 0.868421052631579, 0.7807017543859647, 0.72, 0.7050561797752809, 0.6970443349753693, 0.6981981981981981, 0.6929460580912863, 0.6884615384615385, 0.6879562043795617] 

Curvature Upper Bound
[([1], 0.7298743214511119), ([1, 46], 0.6321205588285577), ([1, 46], 0.6321205588285577), ([1, 46], 0.6321205588285577), ([1, 46], 0.6321205588285577), ([1, 46], 0.6321205588285577), ([1, 46], 0.6321205588285577), ([1, 46], 0.6321205588285577), ([1, 46], 0.6321205588285577), ([1, 46], 0.6321205588285577)] 

IP
[([], 0), ([225], 76), ([225, 428], 132), ([134, 225, 428], 178), ([13

### Small instance: Facility Location on MovieLens

In [6]:
### Example of facility location on MovieLens data

# params
n = 10
k = 5
fname = 'movie'

# open data files and sample
movie = get_data('movie')
data = sample_data(movie, n)
ground_set = set(range(n))

# get obj
obj = get_obj('facility_location', data)

In [7]:
# run greedy
res_gr = run_greedy(k, ground_set, obj, print_log=False)
print('Greedy')
print(res_gr, '\n')

# find optimal solution
res_opt = compute_opt(k, obj, ground_set)
print('OPT')
print(res_opt)

Greedy
[([], 0), ([0], 1.6145065551793651), ([1], 1.6701525059655589), ([6], 1.6772114910918416), ([2], 1.6772114910918416), ([3], 1.6772114910918416)] 

OPT
[([], 0), ([0], 1.6145065551793651), ([0, 1], 1.6701525059655589), ([0, 1, 6], 1.6772114910918416), ([0, 1, 2, 6], 1.6772114910918416), ([0, 1, 2, 3, 6], 1.6772114910918416)]


In [8]:
### run dual

# S_idx specifies the greedy solution sets to comprise mathcal S
ub, approx = compute_dual(k, obj, ground_set, res_gr, S_idx=[0,1,2])
print('Dual')
print('OPT upper bound: ', ub)
print('Greedy approximation: ', approx)

Dual
OPT upper bound:  [1.6145065551793651, 1.6772114910918416, 1.6772114910918416, 1.6772114910918416, 1.6772114910918416]
Greedy approximation:  [1.0, 0.9957912373223203, 1.0, 1.0, 1.0]


In [9]:
### run benchmarks

# run marginal and output OPT upper bound and approximation
ub, approx = compute_marginal(k, res_gr)
print('Marginal')
print('OPT upper bound: ', ub)
print('Greedy approximation: ', approx, '\n')

# run curvature (brute force) and output approximation
res_curv = compute_curvature_brute_k(k, ground_set, obj)
print('Curvature')
print(res_curv, '\n')

# run sharpness output approximation for k = 5 only
apx = compute_s_sharpness_dyn(k, res_opt[-1][0], res_opt[-1][1], ground_set, obj)
print('Sharpness')
print(apx)

Marginal
OPT upper bound:  [1.6145065551793651, 1.6772114910918416, 1.6772114910918416, 1.6772114910918414, 1.6772114910918416]
Greedy approximation:  [1.0, 0.9957912373223203, 1.0, 1.0000000000000002, 1.0] 

Curvature
[0.6321205588285577, 0.6321205588285577, 0.6321205588285577, 0.6321205588285577] 

Sharpness
0.6954789761135712
